In [61]:
from ultralytics.data.dataset import YOLODataset
from ultralytics.data.utils import check_cls_dataset
from ultralytics import YOLO
import requests
import json

In [15]:
ds=YOLODataset("../dataset/batch_01_raw/yolo_format/", data={"names":{
  0: "Cup",
  1: "Timmies"}})

Scanning ../dataset/batch_01_raw/yolo_format/labels.cache... 1402 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1403/1403 [00:00<?, ?it/s]


In [ ]:
from ultralytics.utils.metrics import ConfusionMatrix
from ultralytics.utils.ops import xywh2xyxy
import torch

model0 = YOLO("model0.pt")
model1 = YOLO("model0.pt")

default = "model_0"
models = {
    "model_0": {"model": model0, "describe": {
        "model": "model_0",
        "config": {
            "input_size": [640, 640],
            "batch_size": 16,
            "confidence_threshold": 0.25
        },
        "date_registered": "2025-03-15"
    }},
    "model_1": {"model": model1, "describe": {
        "model": "model_1",
        "config": {
            "input_size": [640, 640],
            "batch_size": 16,
            "confidence_threshold": 0.25
        },
        "date_registered": "2025-03-15"
    }}
}

# IOU THRESHOLD SET AT 45% FOR CALCULATING ACCURACY
model_metrics = {
    "predictions": {
        model_name: {
            "confusion_matrix": ConfusionMatrix(
                nc=2, 
                conf = models[model_name]["describe"]["config"]["confidence_threshold"],
                iou_thres = 0.45,
                task = "detect"),
            "total": 0
        } for model_name in models.keys()
    }
}

def track_accuracy(predictions: list, ground_truth: list,model: str=default):
    """ground_truth takes the form of [{"label": "class id", "bbox": [x, y, w, h]}] and
    predictions is the list of predictions from predict endpoint
    """        
    # Convert predictions to required format (x1, y1, x2, y2, conf, class)
    detections = []
    for pred in predictions:
        box = xywh2xyxy(torch.tensor(pred["bbox"]).view(1, 4)).squeeze()
        class_idx = int(pred["label"].lower()=="timmies")
        detection = torch.cat([
            box,
            torch.tensor([pred["confidence"]]),
            torch.tensor([class_idx])
        ])
        detections.append(detection)
        
    # Convert ground truth boxes to xyxy format
    gt_boxes = []
    gt_cls = []
    for gt in ground_truth:
        box = xywh2xyxy(torch.tensor(gt["bbox"]).view(1, 4)).squeeze()
        gt_boxes.append(box)
        gt_cls.append(gt["label"])
        
    if detections:
        detections = torch.stack(detections)
    else:
        detections = torch.zeros((0, 6))
        
    gt_boxes = torch.stack(gt_boxes) if gt_boxes else torch.zeros((0, 4))
    gt_cls = torch.tensor(gt_cls) if gt_cls else torch.zeros(0)
    
    # Process batch
    model_metrics["predictions"][model]["confusion_matrix"].process_batch(
        detections,
        gt_boxes,
        gt_cls
    )
    
    # update total processed images
    model_metrics["predictions"][model]["total"] += 1
    
    return {"success": True}

In [136]:
for d in ds.get_labels()[:50]:
    predictions=[]
    ground_truth=[]
    resp=requests.post("http://localhost:6132/predict", data={"model":"model_0"}, files={"file":open(d["im_file"],"rb")})
    if resp.ok:
        predictions=resp.json().get("predictions")
    for i in range(len(d["cls"])):
        ground_truth.append({"label":d["cls"][i][0].tolist(), "bbox":d["bboxes"][i].tolist()})
    resp=requests.post("http://localhost:6132/track", json={"model":"model_0", "predictions":predictions,"ground_truth":ground_truth})
    track_accuracy(predictions,ground_truth)


Scanning ../dataset/batch_01_raw/yolo_format/labels.cache... 1402 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1403/1403 [00:00<?, ?it/s]


In [148]:
import pandas
df=pandas.DataFrame(data=model_metrics["predictions"]["model_0"]["confusion_matrix"].matrix)

In [147]:
with open("./offline_batch_results.txt","w") as f:
    f.write(df.to_json(orient="index"))